In [1]:
!unzip Spike-TIMIT.zip

IT/train/dr8/mmpm0/si1061.wav  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1061.wrd  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1061_binary_matrix.mat  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1061_spike_timing.mat  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1691.phn  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1691.txt  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1691.wav  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1691.wrd  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1691_binary_matrix.mat  
  inflating: Spike TIMIT/train/dr8/mmpm0/si1691_spike_timing.mat  
  inflating: Spike TIMIT/train/dr8/mmpm0/si2321.phn  
  inflating: Spike TIMIT/train/dr8/mmpm0/si2321.txt  
  inflating: Spike TIMIT/train/dr8/mmpm0/si2321.wav  
  inflating: Spike TIMIT/train/dr8/mmpm0/si2321.wrd  
  inflating: Spike TIMIT/train/dr8/mmpm0/si2321_binary_matrix.mat  
  inflating: Spike TIMIT/train/dr8/mmpm0/si2321_spike_timing.mat  
  inflating: Spike TIMIT/train/dr8/mmpm0/sx161.phn  
  inflating: Spike TIMIT

In [9]:
from scipy.io import loadmat
# m = loadmat("Spike-TIMIT/test/dr1/faks0/sa1_binary_matrix.mat")



In [57]:
m['binary_output'].shape

(620, 793)

In [52]:
max_time = []
for item in m["spike_output"][0]:
    if len(item)==0:
        continue
    # print(item)
    # print(max(item[0]))
    max_time.append(max(item[0]))
max(max_time)

0.791

In [2]:
import os
def filt_file(path="Spike-TIMIT"):
    # dataset = {}
    train_set = []
    test_set = []
    for d,ds,fs in os.walk("Spike-TIMIT"):
        for fname in fs:
            if "sa" in fname:
                continue
            full_name = d + '/' + fname
            if 'train' in full_name:
                if '.mat' in full_name or '.phn' in full_name:
                  train_set.append(full_name)
            else:
                if '.mat' in full_name or '.phn' in full_name:
                  test_set.append(full_name)
    
    return train_set, test_set

train_set, test_set = filt_file()

In [3]:
def get_pair(file_name_list):
    paris = {}
    for fname in file_name_list:
        id = 0
        pattern = 0
        if '_' in fname:
            id, pattern = fname.split("_",1)
        else:
            id, pattern = fname.split('.')

        # print(fname,id,pattern)
        
        if 'phn' in pattern or 'matrix.mat' in pattern:

            if id not in paris:
                # if 'mat' in pattern or 'phn' in pattern:
                paris[id] = {"phn":fname} if 'phn' in pattern else {"mat":fname}
            else:
                if 'phn' in pattern:
                    paris[id]['phn'] = fname
                else:
                    paris[id]['mat'] = fname
    
    return paris
        

In [5]:
train_file_name_set = get_pair(train_set)
test_file_name_set = get_pair(test_set)

In [18]:
import torch
import tqdm
max_time = 1600
def make_dataset(file_name_set):
    
    def process_file(fmat,fphp):
        m = loadmat(fmat)['binary_output']
        with open(fphp,"r") as fin:
            php_lines = fin.readlines()

        input_pad = torch.cat([torch.ByteTensor(m),torch.ByteTensor(620,max_time-m.shape[1]).fill_(0)],dim=1)
        y = []
        for i, php_line in enumerate(php_lines):
            s,e,p = php_line.split(' ')
            s = int(s)
            e = int(e)
            p_id = p2id[p.strip()]
            y += [p_id] * (int(e/100) - int(s/100))

        input = input_pad.transpose(0,1).unsqueeze(0)
        y += [62] * (max_time - len(y))
        output = torch.LongTensor(y).unsqueeze(0)

        
        return input, output
    i = 0
    for id in tqdm.tqdm_notebook(file_name_set):
        input_tensor, output_tensor= process_file(file_name_set[id]['mat'],file_name_set[id]['phn'])
        # print(input_tensor.shape)
        input_tensors = input_tensor if i == 0 else torch.cat([input_tensors,input_tensor],dim=0)
        output_tensors = output_tensor if i == 0 else torch.cat([output_tensors,output_tensor],dim=0)
        i += 1
        # if i == 10:
        #     break
    
    return input_tensors, output_tensors
        



In [19]:
train_input_tensors, train_output_tensors = make_dataset(train_file_name_set)
test_input_tensors, test_output_tensors = make_dataset(test_file_name_set)

/home/nesa320/anaconda2/envs/contras_sum/lib/python3.6/site-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [21]:
train_input_tensors = train_input_tensors.bool()
train_output_tensors = train_output_tensors.byte()
test_input_tensors = test_input_tensors.bool()
test_output_tensors = test_output_tensors.byte()

In [26]:
perm_idx = torch.randperm(len(train_input_tensors))
train_input_tensors = train_input_tensors[perm_idx]
train_output_tensors = train_output_tensors[perm_idx]
perm_idx = torch.randperm(len(test_input_tensors))
test_input_tensors = test_input_tensors[perm_idx]
test_output_tensors = test_output_tensors[perm_idx]
train_dataset = torch.utils.data.TensorDataset(train_input_tensors[:360],train_output_tensors[:360])
test_dataset = torch.utils.data.TensorDataset(test_input_tensors[:130],test_output_tensors[:130])

In [27]:
torch.save({"train":train_dataset,"test":test_dataset},"timit_processed_data_10_percent.pkl")

In [17]:
train_output_tensors[0]

tensor([39, 39, 39,  ..., 62, 62, 62])

In [164]:
count = 0
for id in train_file_name_set:
    fname = train_file_name_set[id]["phn"]
    with open(fname,"r") as fin:
        a = len(fin.readlines())
    # m = loadmat(fname)["binary_output"]
    # a = m.shape[1]
    # if a > max:
    #     max = a
    count += a

In [168]:
count / len(train_file_name_set) * 620 * 1600 / 10**6

38.35679653679653

In [155]:
max

1514

In [6]:
unique_set = set()
for id in train_file_name_set:
    with open(train_file_name_set[id]['phn'],'r') as fin:
        lines = fin.readlines()
    for line in lines:
        s,e,p = line.split(' ')
        p = p.strip()
        unique_set.add(p)



In [7]:
p2id = {k:i for i,k in enumerate(unique_set)}

In [124]:
p2id

{'ch': 0,
 'v': 1,
 'ah': 2,
 'pcl': 3,
 't': 4,
 'epi': 5,
 'y': 6,
 'p': 7,
 'l': 8,
 'w': 9,
 'nx': 10,
 's': 11,
 'eng': 12,
 'aw': 13,
 'er': 14,
 'q': 15,
 'hh': 16,
 'ih': 17,
 'oy': 18,
 'kcl': 19,
 'ux': 20,
 'el': 21,
 'f': 22,
 'aa': 23,
 'th': 24,
 'iy': 25,
 'bcl': 26,
 'tcl': 27,
 'gcl': 28,
 'k': 29,
 'axr': 30,
 'z': 31,
 'n': 32,
 'r': 33,
 'eh': 34,
 'd': 35,
 'en': 36,
 'sh': 37,
 'jh': 38,
 'ow': 39,
 'dx': 40,
 'pau': 41,
 'ae': 42,
 'ax-h': 43,
 'hv': 44,
 'ao': 45,
 'ng': 46,
 'ax': 47,
 'zh': 48,
 'b': 49,
 'ay': 50,
 'uw': 51,
 'em': 52,
 'm': 53,
 'ey': 54,
 'uh': 55,
 'ix': 56,
 'dh': 57,
 'dcl': 58,
 'h#': 59,
 'g': 60}

In [75]:
len(train_set)/3

3696.0

In [4]:
import numpy as np
path = "N-MNIST/Test/0/00004.bin"
f = open(path, 'rb')
raw_data = np.fromfile(f, dtype=np.uint8)
f.close()

In [6]:
import numpy as np
class Events(object):
    def __init__(self, num_events, width=304, height=240):
        """num_spikes: number of events this instance will initially contain"""
        self.data = np.rec.array(None, dtype=[('x', np.uint16), ('y', np.uint16), ('p', np.bool_), ('ts', np.uint64)], shape=(num_events))
        self.width = width
        self.height = height

def read_events(path):
    # path = "N-MNIST/Test/0/00004.bin"
    f = open(path, 'rb')
    raw_data = np.fromfile(f, dtype=np.uint8)
    f.close()

    all_y = raw_data[1::5]
    all_x = raw_data[0::5]
    all_p = (raw_data[2::5] & 128) >> 7 #bit 7
    all_ts = ((raw_data[2::5] & 127) << 16) | (raw_data[3::5] << 8) | (raw_data[4::5])
    time_increment = 2 ** 13
    overflow_indices = np.where(all_y == 240)[0]
    for overflow_index in overflow_indices:
        all_ts[overflow_index:] += time_increment

    td_indices = np.where(all_y != 240)[0]

    td = Events(td_indices.size, 34, 34)
    td.data.x = all_x[td_indices]
    td.width = td.data.x.max() + 1
    td.data.y = all_y[td_indices]
    td.height = td.data.y.max() + 1
    td.data.ts = all_ts[td_indices]
    td.data.p = all_p[td_indices]

    return td


In [38]:
td.data.ts

array([ 69, 167,  65, ...,  81, 197,  49], dtype=uint64)

In [8]:
all_y.shape

(5293,)

In [15]:
all_p.shape

(5293,)

In [19]:

        # print(full_name)
        # label = int(d[-1])
        # td = read_events(full_name)
        # out = convertd2tensor(td)
        # break
        

/home/nesa320/anaconda2/envs/contras_sum/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
import torch
def convertd2tensor(td):
    max_time = 256
    out_tensor = torch.ByteTensor(max_time,34*34)
    for x,y,p,ts in zip(td.data.x,td.data.y,td.data.p,td.data.ts):
        out_tensor[ts,y*34+x] = int(p)
    return out_tensor
    

In [1]:
import tqdm
import torch
import numpy as np
class Events(object):
    def __init__(self, num_events, width=304, height=240):
        """num_spikes: number of events this instance will initially contain"""
        self.data = np.rec.array(None, dtype=[('x', np.uint16), ('y', np.uint16), ('p', np.bool_), ('ts', np.uint64)], shape=(num_events))
        self.width = width
        self.height = height

def read_events(path):
    # path = "N-MNIST/Test/0/00004.bin"
    f = open(path, 'rb')
    raw_data = np.fromfile(f, dtype=np.uint8)
    f.close()

    all_y = raw_data[1::5]
    all_x = raw_data[0::5]
    all_p = (raw_data[2::5] & 128) >> 7 #bit 7
    all_ts = ((raw_data[2::5] & 127) << 16) | (raw_data[3::5] << 8) | (raw_data[4::5])
    time_increment = 2 ** 13
    overflow_indices = np.where(all_y == 240)[0]
    for overflow_index in overflow_indices:
        all_ts[overflow_index:] += time_increment

    td_indices = np.where(all_y != 240)[0]

    td = Events(td_indices.size, 34, 34)
    td.data.x = all_x[td_indices]
    td.width = td.data.x.max() + 1
    td.data.y = all_y[td_indices]
    td.height = td.data.y.max() + 1
    td.data.ts = all_ts[td_indices]
    td.data.p = all_p[td_indices]

    return td


def convertd2tensor(td):
    max_time = 256
    out_tensor = torch.ByteTensor(max_time,34*34).fill_(0)
    for x,y,p,ts in zip(td.data.x,td.data.y,td.data.p,td.data.ts):
        out_tensor[ts,y*34+x] = int(p)
    return out_tensor

def make_dataset(split):
# split = "Test"
    f_list = []
    for d,ds,fs in tqdm.tqdm_notebook(os.walk("N-MNIST/"+split)):
        for fname in fs:
            full_name = d + '/' + fname
            f_list.append(full_name)
    print(len(f_list))
    input_tensor = torch.ByteTensor(len(f_list),256,34*34)
    y = torch.LongTensor(len(f_list))
    i = 0
    for f in tqdm.tqdm_notebook(f_list):
        d,a = f.split(split)
        label = int(a[1])
        td = read_events(f)
        input_tensor[i] = convertd2tensor(td)
        y[i] = label
        i += 1
    dataset = torch.utils.data.TensorDataset(input_tensor,y)
    return dataset

    

In [2]:
train_dataset = make_dataset("Train")
test_dataset = make_dataset("Test")

/home/nesa320/anaconda2/envs/contras_sum/lib/python3.6/site-packages/ipykernel_launcher.py:49: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



60000
/home/nesa320/anaconda2/envs/contras_sum/lib/python3.6/site-packages/ipykernel_launcher.py:57: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



10000


In [8]:
torch.save([train_dataset, test_dataset],"processed_n_mnist.pkl")

In [6]:
import torch
train_dataset, test_dataset = torch.load("processed_n_mnist.pkl")

In [13]:
train_dataset[3][0].sum()

tensor(2495)

In [5]:
train_dataset[0][0].sum()

tensor(2013)

In [20]:
import torch
train_dataset, test_dataset = torch.load("processed_n_mnist.pkl")

In [4]:
import torch
acc = torch.load("log/lenet_conv_with_modified_actfun.pkl")

In [5]:
acc["acc"]

[79.4,
 82.79,
 86.05,
 85.2,
 86.11,
 87.19,
 86.64,
 87.74,
 88.11,
 87.68,
 87.75,
 88.53,
 89.01,
 89.07,
 89.16,
 89.36,
 88.72,
 88.94,
 89.32,
 89.14,
 89.27]